In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
df3 = pd.read_csv("/content/df3.csv")

<ipython-input-2-4040a7e3668a>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("/content/df3.csv")


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   gender                                      36457 non-null  int64  
 1   car                                         36457 non-null  int64  
 2   reality                                     36457 non-null  int64  
 3   work_phone                                  36457 non-null  int64  
 4   phone                                       36457 non-null  int64  
 5   email                                       36457 non-null  int64  
 6   family_size                                 36457 non-null  float64
 7   begin_month                                 36457 non-null  float64
 8   credit                                      36457 non-null  object 
 9   employed_age                                36457 non-null  float64
 10  age       

##데이터처리

In [ ]:
train = df3[:26457]
test = df3[26457:].reset_index(drop=True)

test.drop(labels=["credit"],axis=1, inplace=True)

In [ ]:
train["credit"] = train["credit"].astype(float).astype(int)

<ipython-input-6-cd75d914b685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["credit"] = train["credit"].astype(float).astype(int)


In [ ]:
train_X=train.drop('credit', axis=1)
train_Y=train[['credit']]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_X, train_Y,  stratify=train_Y, test_size=0.25,random_state = 1)

## 스태킹 앙상블
- 선례 조사와 gridsearch 를 토대로 찾은 베스트 하이퍼 파라미터로 학습

**베이스모델**

In [ ]:
# 모델 라이브러리 임포트
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
!pip install catboost
from catboost import CatBoostClassifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.5 MB/s eta 0:00:00


In [ ]:

seed = 42
n_class = 3

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': 500,
    'objective': 'multiclass',
    'random_state': seed,
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 150,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'num_class': n_class
}

ctb_params = {
    'random_state':42,
    'iterations':1000,
    'grow_policy':'Lossguide',
    'learning_rate':0.01,
    'reg_lambda':0.095,
    'max_depth':20,
    'num_leaves':300,
    'min_child_samples':10,
    'max_bin':300
}

rf_params = {
    'max_depth': 30,
    'criterion':"entropy",
    'min_samples_leaf': 3,
    'random_state': seed,
    'n_estimators': 500
}

In [ ]:
# 모델 생성
lgb_model = LGBMClassifier(**lgb_params)
rf_model = RandomForestClassifier(**rf_params)
ctb_model = CatBoostClassifier(**ctb_params)

# 기반 모델 리스트 생성
base_models = [('lgb', lgb_model), ('rf', rf_model), ('ctb', ctb_model)]

**앙상블**

In [ ]:
from sklearn.metrics import log_loss
from sklearn.ensemble import StackingClassifier

# 메타 모델로 LightGBM 사용
meta_model = LGBMClassifier(n_estimators=100, learning_rate=0.01)

# 스태킹 앙상블 모델 구성
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=15)

# 스태킹 앙상블 모델 학습
stacking_model.fit(X_train, y_train)

# 검증 세트에 대한 레이블별 확률 값 예측
val_pred_proba = stacking_model.predict_proba(X_val)

# Log Loss 값 계산
val_log_loss = log_loss(y_val, val_pred_proba)
print(f'Validation Log Loss: {val_log_loss}')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5063:	learn: 0.1116816	total: 7m 18s	remaining: 7m 7s
5064:	learn: 0.1116557	total: 7m 18s	remaining: 7m 7s
5065:	learn: 0.1116353	total: 7m 18s	remaining: 7m 7s
5066:	learn: 0.1116107	total: 7m 18s	remaining: 7m 6s
5067:	learn: 0.1115856	total: 7m 18s	remaining: 7m 6s
5068:	learn: 0.1115637	total: 7m 18s	remaining: 7m 6s
5069:	learn: 0.1115387	total: 7m 18s	remaining: 7m 6s
5070:	learn: 0.1115167	total: 7m 18s	remaining: 7m 6s
5071:	learn: 0.1114925	total: 7m 18s	remaining: 7m 6s
5072:	learn: 0.1114676	total: 7m 19s	remaining: 7m 6s
5073:	learn: 0.1114465	total: 7m 19s	remaining: 7m 6s
5074:	learn: 0.1114239	total: 7m 19s	remaining: 7m 6s
5075:	learn: 0.1113990	total: 7m 19s	remaining: 7m 6s
5076:	learn: 0.1113773	total: 7m 19s	remaining: 7m 6s
5077:	learn: 0.1113576	total: 7m 19s	remaining: 7m 5s
5078:	learn: 0.1113329	total: 7m 19s	remaining: 7m 5s
5079:	learn: 0.1113130	total: 7m 19s	remaining: 7m 5s
5080:	learn: 0.1112889	total: 7m 19s	remaining

In [ ]:
pred = stacking_model.predict_proba(test)
#pred

In [ ]:
sub = pd.read_csv('./sample_submission.csv')

In [ ]:
sub['0']=pred[:,0]
sub['1']=pred[:,1]
sub['2']=pred[:,2]
#sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)